# Deep Convolutional Conditional Generative Adversarial Networks (DCCGAN)

A model skeleton for Conditional Generative Adversarial Networks (GAN)s. It was first introduced in the paper titled [Conditional Generative Adversarial Nets](https://arxiv.org/abs/1411.1784) by Mehdi Mirza and Simon Osindero.

> The model is a variant of the original GAN model, where the generator and discriminator are modified to take an additional input, the class label, which is a one-hot vector. This label is incorporated in both the generator and discriminator, such that the generated images are conditioned on the input label. The model can be used to generate images of a specific class.

## usage

In [1]:
import deeply
import deeply.datasets as dd

mnist, info = dd.load("mnist", shuffle_files = True, as_supervised = True, with_info = True)
features    = info.features
image_shape = features["image"].shape
n_classes   = features["label"].num_classes

gan = deeply.hub("dcgan", input_shape = image_shape, n_classes = n_classes,
                 decoder_batch_norm = True, encoder_dropout_rate = 0.3,
                 encoder_layer_growth_rate = 2, init_decoder_units = 256,
                 decoder_layer_growth_rate = 0.5, kernel_size = 5,
                 decoder_strides = [1, 2], final_activation = "tanh")

2022-09-25 02:02:24.851161: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-25 02:02:27.536542: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## example

In [2]:
import tensorflow as tf
from deeply.plots import imgplot as imgplt, mplt
from deeply.transformers.scaler import image_scaler
from tqdm.auto import tqdm, trange

def mapper(image, label):
    return image_scaler.fit_transform(image), tf.one_hot(label, n_classes)

config = dict(batch_size = 256,
              epochs = 50)

data   = mnist["train"].map(mapper)
batch  = data.batch(config["batch_size"])

In [3]:
from deeply.model.gan import plot_img_samples

for e in trange(config["epochs"]):
    for b in batch:
        history = gan.fit(b, checkpoint_path = "gan.h5d5")
    plot_img_samples(gan)

  0%|          | 0/50 [00:00<?, ?it/s]

2022-09-25 02:02:28.264360: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


ValueError: in user code:

    File "/home/helikarlab/.venv/lib/python3.10/site-packages/keras/engine/training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "/home/helikarlab/.venv/lib/python3.10/site-packages/keras/engine/training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/helikarlab/.venv/lib/python3.10/site-packages/keras/engine/training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "/home/helikarlab/dev/deeply/src/deeply/model/gan.py", line 99, in train_step
        loss_generator, loss_discriminator = self.compute_loss(data)
    File "/home/helikarlab/dev/deeply/src/deeply/model/gan.py", line 86, in compute_loss
        real_output      = self.discriminator(data, training = True)
    File "/home/helikarlab/.venv/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_t